In [1]:
"""
Jupyter Notebook code for a 3-layer neural network for classifing the MNIST dataset using the PyTorch framework.
PyTorch framework which allows GPU acceleration if available.

From "Learning MNIST with GPU Acceleration - A Step by Step PyTorch Tutorial" by Tariq Rashid, May 2017
http://makeyourownneuralnetwork.blogspot.com/2017/05/learning-mnist-with-gpu-acceleration.html
https://github.com/makeyourownneuralnetwork/pytorch

(c) Tariq Rashid, 2017
license is GPLv2
"""

import torch
from torch.autograd import Variable
import torch.nn as nn

import numpy
import scipy.special       # scipy.special for the sigmoid function expit()
import matplotlib.pyplot   # library for plotting arrays
import time                # for calculating run time

# ensure the plots are inside this notebook, not an external window
%matplotlib inline

ImportError: No module named torch

In [ ]:
class NeuralNetwork(nn.Module):

    def __init__(self, inodes, hnodes, onodes, learning_rate):
        # call the base class's initialisation too
        super().__init__()
        
        # dimensions
        self.inodes = inodes
        self.hnodes = hnodes
        self.onodes = onodes
        
        # learning rate
        self.lr = learning_rate
        
        # define the layers and their sizes, turn off bias
        self.linear_ih = nn.Linear(inodes, hnodes, bias=False)
        self.linear_ho = nn.Linear(hnodes, onodes, bias=False)
        
        # define activation function
        self.activation = nn.Sigmoid()
        
        # create error function
        self.error_function = torch.nn.MSELoss(size_average=False)

        # create optimiser, using simple stochastic gradient descent
        self.optimiser = torch.optim.SGD(self.parameters(), self.lr)

        pass

    
    def forward(self, inputs_list):
        """convert list to a 2-D FloatTensor then wrap in Variable"""
        
        # also shift to GPU, remove .cuda. if not desired
        inputs = Variable(torch.cuda.FloatTensor(inputs_list).view(1, self.inodes))
        
        # combine input layer signals into hidden layer
        hidden_inputs = self.linear_ih(inputs)
        
        # apply sigmiod activation function
        hidden_outputs = self.activation(hidden_inputs)
        
        # combine hidden layer signals into output layer
        final_inputs = self.linear_ho(hidden_outputs)
        
        # apply sigmiod activation function
        final_outputs = self.activation(final_inputs)
        
        return final_outputs

    
    def train(self, inputs_list, targets_list):
        # calculate the output of the network
        output = self.forward(inputs_list)

        # create a Variable out of the target vector, doesn't need gradients calculated
        # also shift to GPU, remove .cuda. if not desired
        target_variable = Variable(torch.cuda.FloatTensor(targets_list).view(1, self.onodes), requires_grad=False)
        
        # calculate error
        loss = self.error_function(output, target_variable)
        #print(loss.data[0])

        # zero gradients, perform a backward pass, and update the weights.
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()
        pass

    pass

In [ ]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate
learning_rate = 0.2

# create instance of neural network
n = NeuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

# move neural network to the GPU, delete if not desired
n.cuda()

In [ ]:
# load the mnist training data CSV file into a list
training_data_file = open("Data Sets/MNIST Dataset/mnist_train_100.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [ ]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 100

start = time.time()
for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

end = time.time()
print("Training time took {:.5} seconds".format(end - start))

In [ ]:
# load the mnist test data CSV file into a list
test_data_file = open("Data Sets/MNIST Dataset/mnist_test_10.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [ ]:
# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    
    # correct answer is first value
    correct_label = int(all_values[0])
    
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    
    # query the network
    outputs = n.forward(inputs)
    
    # the index of the highest value corresponds to the label
    m, label = outputs.max(1)
    
    # append correct or incorrect to list
    # need to extract from pytorch tensor via numpy to compare to python integer
    if (label.data[0][0] == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

In [ ]:
# calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

# Sources
* [Learning MNIST with GPU Acceleration - A Step by Step PyTorch Tutorial](http://makeyourownneuralnetwork.blogspot.com/2017/05/learning-mnist-with-gpu-acceleration.html)
* [Python code](https://github.com/makeyourownneuralnetwork/pytorch)
* [Install PyTorch on Ubuntu](https://www.youtube.com/watch?v=PSC0RVcubnA)
* [Torch Video Tutorials](https://github.com/Atcold/torch-Video-Tutorials)
* [How Pytorch gives the big picture with deep learning](https://medium.freecodecamp.org/how-pytoch-gives-the-big-picture-with-deep-learning-e4a0f372f4b6)